<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/AdiabaticFlash_ScipyRoot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone --depth 1 https://github.com/profteachkids/chetools.git &> /dev/null
import sys
sys.path.insert(1, "/content/chetools") #Path to CHE module imports

In [32]:
import numpy as np
from scipy.optimize import root

In [11]:
import tools.che as che
R=8.314 # J/(mol K)

In [12]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [26]:
def xtov(x):
    return np.log(x[:-1]) - np.log(x[-1])

def vtox(v):
    x = np.empty(v.size+1)
    x[:-1]=np.exp(v)/(1+np.sum(np.exp(v)))
    x[-1]=1-np.sum(x[:-1])
    return x

In [29]:
Ftot=10 # Total Feed moles
Fz = np.array([1/3, 1/3, 1/3]) # Equimolar feed composition
ncomp = Fz.size
FT = 450 # Feed temperature
flashP= 101325 # Flash drum pressure

Vyv = xtov(Fz) # Guess vapor/liquid composition equal to feed
Lxv = xtov(Fz) # Comp - constrains mole fractions to behave like mole fractions!
flashT = np.array([360.])  # Guess and bounds for flash temperature
Vtot = np.array([Ftot/2])  # Guess half of feed in vapor
Ltot = np.array([Ftot/2])

x_guess = jnp.concatenate([Vyv, Lxv, flashT, Vtot, Ltot])

In [30]:
def model(x):
  Vy = vtox(x[0:ncomp-1])
  Lx = vtox(x[ncomp-1:2*(ncomp-1)])
  flashT, Vtot, Ltot = x[-3:]

  V = Vy * Vtot
  L = Lx * Ltot
  F = Fz * Ftot
  mass_balance = F-V-L

  FH = p.Hl(nL=F, T=FT)
  VH = p.Hv(nV=V, T=flashT)
  LH = p.Hl(nL=L, T=flashT)
  energy_balance = jnp.array([FH - VH - LH])

  fugL = Lx  * p.NRTL_gamma(Lx,flashT)* p.Pvap(flashT)
  fugV = Vy*flashP
  VLE = fugL - fugV
  return jnp.concatenate([mass_balance, energy_balance, VLE])

In [36]:
sol=root(model,x_guess).x
Vy = vtox(sol[0:ncomp-1])
Lx = vtox(sol[ncomp-1:2*(ncomp-1)])
flashT, Vtot, Ltot = sol[-3:]

In [37]:
print(Vy)
print(Lx)
print(flashT, Vtot, Ltot)

[0.35170977 0.3396212  0.30866903]
[0.32122647 0.32919072 0.34958281]
352.8549750577987 3.9716387833198716 6.028361216680128


In [4]:
v = np.log(x[:-1]) - np.log(x[-1])

In [5]:
v

array([-1.38629436, -0.69314718, -0.28768207])

In [8]:
xx = np.exp(v)/(1+np.sum(np.exp(v)))

In [9]:
xx

array([0.1, 0.2, 0.3])